In [45]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import sklearn
metric = sklearn.metrics.accuracy_score

In [46]:
'''
from sklearn.model_selection import StratifiedShuffleSplit

stratSplit = StratifiedShuffleSplit(labels, test_size=0.5, random_state=42)
for train_idx, test_idx in stratSplit:
    X_train=data[train_idx]
    y_train=labels[train_idx]
'''

'\nfrom sklearn.model_selection import StratifiedShuffleSplit\n\nstratSplit = StratifiedShuffleSplit(labels, test_size=0.5, random_state=42)\nfor train_idx, test_idx in stratSplit:\n    X_train=data[train_idx]\n    y_train=labels[train_idx]\n'

In [63]:
np.sum(data['v1'] == 1)

747

In [64]:
n = 747
sample_yes = data.ix[data.v1 == 1].sample(n=n, replace=False, random_state=0)
sample_no = data.ix[data.v1 == 0].sample(n=n, replace=False, random_state=0)
data = pd.concat([sample_yes, sample_no])
data = data.sample(frac=1).reset_index(drop=True)

# Sort the over-represented class to the head.
#labels = labels[labels.apply(len).sort_values(ascending=False).index]
#excess = len(labels.iloc[0]) - len(labels.iloc[1])
#remove = np.random.choice(labels.iloc[0], excess, replace=False)
#data = data[~data.name.isin(remove)]

'''
idx = np.arange(len(data.v1))
np.random.shuffle(idx)
# train on a random 2/3 and test on the remaining 1/3
idx_train = idx[:2*len(idx)//3]
idx_test = idx[2*len(idx)//3:]
X_train = data[idx_train]
X_test = data[idx_test]
y_train = labels[idx_train]
y_test = labels[idx_test]

'''


/home/vasilis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/vasilis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


'\nidx = np.arange(len(data.v1))\nnp.random.shuffle(idx)\n# train on a random 2/3 and test on the remaining 1/3\nidx_train = idx[:2*len(idx)//3]\nidx_test = idx[2*len(idx)//3:]\nX_train = data[idx_train]\nX_test = data[idx_test]\ny_train = labels[idx_train]\ny_test = labels[idx_test]\n\n'

In [49]:
'''
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

c1 = np.random.rand(50, 2)/5
c2 = (-0.6, 0.5) + np.random.rand(50, 2)/5
data = np.concatenate((c1, c2))
labels = np.array([0] * 50 + [1] *50)
plt.figure(figsize=(6, 6))
plt.subplot(111, xticks=[], yticks=[])
plt.scatter(data[:50, 0], data[:50, 1], color='navy')
plt.scatter(data[50:, 0], data[50:, 1], color='c')
idx = np.arange(len(labels))
np.random.shuffle(idx)
# train on a random 2/3 and test on the remaining 1/3
idx_train = idx[:2*len(idx)//3]
idx_test = idx[2*len(idx)//3:]
X_train = data[idx_train]
X_test = data[idx_test]
y_train = labels[idx_train]
y_test = labels[idx_test]
'''

"\nimport matplotlib.pyplot as plt\nimport numpy as np\nnp.random.seed(0)\n\nc1 = np.random.rand(50, 2)/5\nc2 = (-0.6, 0.5) + np.random.rand(50, 2)/5\ndata = np.concatenate((c1, c2))\nlabels = np.array([0] * 50 + [1] *50)\nplt.figure(figsize=(6, 6))\nplt.subplot(111, xticks=[], yticks=[])\nplt.scatter(data[:50, 0], data[:50, 1], color='navy')\nplt.scatter(data[50:, 0], data[50:, 1], color='c')\nidx = np.arange(len(labels))\nnp.random.shuffle(idx)\n# train on a random 2/3 and test on the remaining 1/3\nidx_train = idx[:2*len(idx)//3]\nidx_test = idx[2*len(idx)//3:]\nX_train = data[idx_train]\nX_test = data[idx_test]\ny_train = labels[idx_train]\ny_test = labels[idx_test]\n"

In [50]:
#pd.DataFrame(data.values)

In [51]:

train_data = data[:500]
test_data = data[500:]


In [52]:
#Vectorize the text

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data.v2)
y_train = train_data.v1

X_test = vectorizer.transform(test_data.v2)
y_test = test_data.v1


In [53]:
from sklearn.linear_model import Perceptron
model_1 = Perceptron(max_iter=1000, tol=1e-3)
model_1.fit(X_train, y_train)
print('accuracy (train): %5.2f'%(metric(y_train, model_1.predict(X_train))))
print('accuracy (test): %5.2f'%(metric(y_test, model_1.predict(X_test))))

model_2 = RandomForestClassifier(n_estimators=20, n_jobs=-1)
model_2.fit(X_train, y_train)
print('accuracy (train): %5.2f'%(metric(y_train, model_2.predict(X_train))))
print('accuracy (test): %5.2f'%(metric(y_test, model_2.predict(X_test))))

from sklearn.ensemble import AdaBoostClassifier
model_3 = AdaBoostClassifier(n_estimators=3)
model_3.fit(X_train, y_train)
print('accuracy (train): %5.2f'%(metric(y_train, model_3.predict(X_train))))
print('accuracy (test): %5.2f'%(metric(y_test, model_3.predict(X_test))))

accuracy (train):  1.00
accuracy (test):  0.93
accuracy (train):  1.00
accuracy (test):  0.93
accuracy (train):  0.79
accuracy (test):  0.78


In [54]:
# Define Classifier

#ensemble = RandomForestClassifier(n_estimators=10, n_jobs=-1).fit(vectorized_text, train_data.v1)
ensemble = [model_1,model_2,model_3]
n_models = len(ensemble)

predictions = np.array([h.predict(X_train) for h in ensemble])
# scale hij to [-1/N, 1/N]
predictions =(predictions* 1/n_models)

λ = 1


In [55]:
w = predictions @ predictions.T
wii = X_train.shape[0] / (n_models ** 2) + λ - 2 * predictions @ y_train
w[np.diag_indices_from(w)] = wii
W = {}
for i in range(n_models):
    for j in range(i, n_models):
        W[(i, j)] = w[i, j]

In [56]:
import dimod
#sampler = dimod.SimulatedAnnealingSampler()
#response = sampler.sample_qubo(W, num_reads=100)
#weights = list(response.first.sample.values())

In [57]:
h, J, offset = dimod.qubo_to_ising(W)
from qiskit.quantum_info import Pauli
from qiskit.aqua import Operator

num_nodes = len(w)
pauli_list = []
for i in range(num_nodes):
    wp = np.zeros(num_nodes)
    vp = np.zeros(num_nodes)
    vp[i] = 1
    pauli_list.append([h[i], Pauli(vp, wp)])
    for j in range(i+1, num_nodes):
        if w[i, j] != 0:
            wp = np.zeros(num_nodes)
            vp = np.zeros(num_nodes)
            vp[i] = 1
            vp[j] = 1
            pauli_list.append([J[i, j], Pauli(vp, wp)])
ising_model = Operator(paulis=pauli_list)

/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:62: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)
/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:369: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)
/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:267: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)


In [58]:
from qiskit.aqua import get_aer_backend, QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
p = 1
optimizer = COBYLA()
qaoa = QAOA(ising_model, optimizer, p, operator_mode='matrix')
backend = get_aer_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=100)
result = qaoa.run(quantum_instance)

/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:200: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)
/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operators/op_converter.py:92: DeprecationWarning: The `Operator` class is deprecated. Please use `WeightedPauliOperator` or `TPBGroupedWeightedPauliOperator` or `MatrixOperator` instead
  DeprecationWarning)
/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:200: DeprecationWarning: The `Operator` class is deprecated and will be removed after 0.6. Use the class for each representation instead, including `MatrixOperator`, `WeightedPauliOperator` and `TPBGroupedWeightedPauliOperator`
  DeprecationWarning)
/home/vasilis/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operator.py:

In [59]:
def predict(models, weights, X):

    n_data = X.shape[0]
    T = 0
    y = np.zeros(n_data)
    for i, h in enumerate(models):
        #print('type of predict')
        y0 = weights[i] * h.predict(X)  # prediction of weak classifier
        y += y0
        T += np.sum(y0)

    y = np.sign(y - T / (n_data*len(models)))

    return y

In [69]:
k = np.argmin(result['eigvecs'][0])
weights = np.zeros(num_nodes)
for i in range(num_nodes):
    weights[i] = k % 2
    k >>= 1

In [70]:
weights

array([1., 1., 1.])

In [71]:
#vectorized_test = vectorizer.transform(test_data.v2)
print('accuracy (test): %5.2f'%(metric( y_test, predict(ensemble, weights, X_test))))
print('accuracy (train): %5.2f'%(metric( y_train, predict(ensemble, weights, X_train))))
#print('accuracy (test): %5.2f'%(metric( test_data.v1, predict(ensemble, weights, vectorized_text))))

accuracy (test):  0.49
accuracy (train):  0.51


SyntaxError: invalid syntax (<ipython-input-65-f5469ce6e0b8>, line 2)

In [74]:
%%latex
$$\hat{H}$$

<IPython.core.display.Latex object>